In [13]:
import re
from selenium.webdriver import PhantomJS
import time
from http.client import RemoteDisconnected
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import string
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException,NoSuchElementException,WebDriverException
import nltk
import pymysql
from pymysql import InternalError,ProgrammingError
import itertools

In [14]:
from abc import abstractmethod, ABC


class Scraper(ABC):
    def __init__(self, start_link, dynamic=False):

        self.browser = self.get_crawler(dynamic)
        self.db_executer = self.connect_to_database()
        self.start_link = start_link
        self.temporary_memory = set()

    def connect_to_database(self):
        global conn
        global cur
        conn = pymysql.connect(host='35.198.176.76', user='root', passwd=None, db='mysql', charset='utf8')
        cur = conn.cursor()
        cur.execute("USE definitions")
        return cur

    def get_crawler(self, dynamic):
        crawler = None
        if (dynamic):
            crawler = PhantomJS("/Users/mac/Desktop/Web scraping/phantomjs-2.1.1-macosx/bin/phantomjs")
        return crawler

    @abstractmethod
    def scraping_strategy(self):
        pass

    def insert_to_database(self, row):

        term, definition, example = row
        if example is None:
            example = ""

        cur = self.db_executer
        cur.execute("SELECT definitions,examples FROM terms WHERE term = '{}'".format(term))
        res = cur.fetchall()
        try:
            examples_alr= res[0][1]
            definitions_alr = res[0][0]
        except IndexError:
            
            examples_alr=""
            definitions_alr=""
        
        if len(definitions_alr)>1:
            definitions_alr = [defin.replace("'","`") for defin in definitions_alr]
            definitions_alr = "".join(definitions_alr)
            
        print("Already in database - ", res)
        
        try:
            if (len(res) > 0):

                if definition not in res[0][0]:

                        cur.execute("UPDATE terms SET definitions = '{0} &$ {1}',examples = '{2} &$ {3}' WHERE term = '{4}'".format(
                           definitions_alr, definition,examples_alr, example, term))

            else:
                query = "INSERT into terms (term,definitions,examples) VALUES ('{0}','{1}','{2}')".format(term, definition,
                                                                                                          example)
                cur.execute(query)

            conn.commit()
        except ProgrammingError:
            print("Programming Error")
            print(definitions_alr)

    def finalize(self):
        cur.close
        conn.close()

    def find_pos_tag(self, sentence, needed_word):
        pattern = re.compile(r"" + needed_word + r"e?s?", re.M | re.I)
        tokenizer = nltk.word_tokenize
        pos_tags = nltk.pos_tag(tokenizer(sentence))
        needed_tag = [y for x, y in pos_tags if len(re.findall(pattern, x)) > 0]
        try:
            needed_tag = needed_tag[0]
        except IndexError:
            needed_tag = ""
        return needed_tag

    def handle_internal_error(self):
        cur = self.db_executer
        queries = ["SHOW FULL PROCESSLIST"]
        [cur.execute(query) for query in queries]
        results = (cur.fetchall())
        thread_ids = [result[0] for result in results]
        print(thread_ids)
        
        queries = ["KILL " + str(thrd_id) for thrd_id in thread_ids]
        print(queries)
        [cur.execute(query) for query in queries]
        self.db_executer = self.connect_to_database





In [15]:

class VocabularyScraper(Scraper):
    def __init__(self, *args, **kwargs):
        super(self.__class__, self).__init__(*args, **kwargs)

    def preprocess_exs_defs(self, term):
        webdriver = self.browser

        term = term.replace("'", "`")

        definitions = list(itertools.chain(*[webdriver.find_elements_by_css_selector(name) for name in
                                             [".wordPage .blurb .short", ".wordPage .blurb .long",
                                              ".wordPage .definition h3"]]))

        examples = webdriver.find_elements_by_css_selector(".wordPage .example")

        definitions = self.remove_redund_chars(definitions)
        examples = self.remove_redund_chars(examples)

        examples = " &$ ".join(['&%s& %s' % (self.find_pos_tag(example, term), example) for example in examples])

        definitions = " &$ ".join([definition for definition in definitions])

        row = tuple((term, definitions, examples))
        return row

    def remove_redund_chars(self, iterable):

        if iterable is not None:
            filtered = [element.text.replace("'", '`') for element in iterable]
            return filtered

    def enter_search_field(self, key):
        webdriver = self.browser
        search_field = webdriver.find_element_by_id("search")
        search_field.clear()
        search_field.send_keys(key)

        # search_field.send_keys(Keys.ENTER)



        WebDriverWait(webdriver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "entry")))

    def preparation(self, key):
        found = False
        while not found:
            try:
                webdriver = self.browser
                webdriver.get(self.start_link)
                self.enter_search_field(key)
                found = True
            except  TimeoutException:
                print("Exceeded time.. Retrying to get search field")

    def load_more(self):
        webdriver = self.browser
        div = webdriver.find_element_by_class_name("hasmore")
        webdriver.execute_script("arguments[0].scrollTop += arguments[0].scrollHeight", div)

    def scraping_strategy(self):
        
        letter = "i"
        self.preparation(letter)
        webdriver = self.browser
        count=0

        # Just to initialize for checking 
        
        while True:
            try:
                self.load_more()
                print(count)
                count+=20
                time.sleep(0.2)
            except NoSuchElementException:
                print("retrying....")
                
                time.sleep(2)
                try:
                    self.load_more()
                    self.browser.save_screenshot("screen.png")
                    self.page_source = self.browser.page_source
                except NoSuchElementException:
                    break

        

       
        words = webdriver.find_elements_by_css_selector(".autocomplete .word")
        print("Words were scraped ",len(words))
        
        time_started = time.time() 

        for word in words:
            try:

                term = word.get_attribute("innerHTML")
                
                time_started = time.time()
                word.click()
                time.sleep(1)

                print("Term is ", term)
                
                row = self.preprocess_exs_defs(term)

                print(row)

                self.insert_to_database(row)
                time_ended = time.time()
                print("{}s  took to scrape {}".format(round(time_ended-time_started,5),term))

            except StaleElementReferenceException:

                print("Word %s was not loaded into dataset" % term)
                continue


            except InternalError:
                print("Trying to handle multiple threading error")
                self.handle_internal_error()


           # except ProgrammingError:
           #     print("Programming error")
           #     self.finalize()
           #     self.db_executer = self.connect_to_database()


            time.sleep(.5)

            
        time_ended = time.time()

        print("Scraping of the letter '{}' was successfuly finished,it took it {} seconds".format(letter,
                                                                                                  time_ended - time_started))
        self.finalize()


test = VocabularyScraper("https://www.vocabulary.com/dictionary/", dynamic=True)

In [16]:
test.scraping_strategy()

Exceeded time.. Retrying to get search field
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500


KeyboardInterrupt: 

In [16]:
test.db_executer.execute("SELECT * FROM terms WHERE term LIKE 'a%' ")

19943

In [7]:
result[::-1]

((81094, 'Zyrian', 'n\nthe Finnic language spoken by the Komi', ''),
 (81097,
  'zymurgy',
  'n\nthe branch of chemistry concerned with fermentation (as in making wine or brewing or distilling)',
  ''),
 (81100,
  'zymotic',
  'adj\nof or relating to or causing fermentation &$ adj\nrelating to or caused by infection',
  ''),
 (81103,
  'zymosis',
  'n\na process in which an agent causes an organic substance to break down into simpler substances; especially, the anaerobic breakdown of sugar into alcohol &$ n\n(medicine) the development and spread of an infectious disease (especially one caused by a fungus)',
  ''),
 (81106, 'zymolytic', 'adj\nof or relating to or causing fermentation', ''),
 (81109,
  'zymolysis',
  'n\na process in which an agent causes an organic substance to break down into simpler substances; especially, the anaerobic breakdown of sugar into alcohol',
  ''),
 (81112,
  'zymology',
  'n\nthe branch of chemistry concerned with fermentation (as in making wine or brewin

In [18]:
result=test.db_executer.fetchall()

In [28]:
swelter = [x for x in result if x[1]=="abrupt"]
swelter

[(907,
  'abrupt',
  'extremely steep &$ marked by sudden changes in subject and sharp transitions &$ To interrupt suddenly &$ To tear off or asunder &$ exceedingly sudden and unexpected &$ Without notice to prepare the mind for the event; sudden; hasty; unceremonious &$ Curt in manner; rude; uncivil; impolite &$ surprisingly and unceremoniously brusque in manner &$ Something which is abrupt &$ Extremely steep or craggy as if broken up; precipitous &$ Suddenly terminating, as if cut off; truncate &$ Having sudden transitions from one subject or state to another; unconnected &$ An abrupt decision is one you make suddenly. An abrupt halt is one the Road Runner comes to when reaching an abrupt drop-off—you`d hear brakes squealing. &$ Because being abrupt with someone is often a way of showing them that you are angry, you often apologize for the appearance of it. "I`m sorry I was abrupt. I was rushing to get to class." The root of abrupt is Latin rumpere "to break," which also gives us rup